In [ ]:
import re
from typing import Any

class User:
    def __init__(self, user_id: str, name: str, email: str) -> None:
        self.user_id = user_id
        self.name = name
        self.email = email

    def registration(self) -> None:
        if (self.__check_user_exists()):
            return print(f"{self.name} registration already exits.")
        else:
            with open("users.txt", "a") as user_records:
                user_records.write(f"{self.user_id},{self.name},{self.email}\n")
                user_records.close()


    def __check_user_exists(self) -> bool:
        with open("users.txt", "r") as user_records:
            for line in user_records:
                if self.name in line:
                    return True
        return False

    @staticmethod
    def validate_user_email(email) ->  bool:
        valid = re.match(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', email)
        return bool(valid)


class Librarian(User):
    def __init__(self, user_id: str, name: str, email: str):
        super().__init__(user_id, name, email)
        self.registration()

    def add(self, book_id: str, title: str, author: str, availability:bool) -> bool:
        if self.__check_book_exists(title):
            return False
        else:
            new_book = Book(book_id, title, author, availability=True)
            self.__save_book_to_file(new_book)
            return True

    def __save_book_to_file(self, book: Any) -> None:
        with open("books.txt", "a") as book_records:
            book_records.write(f"{book.book_id},{book.title},{book.author},{book.availability}\n")
            book_records.close()

    def __check_book_exists(self, title: str) -> bool:
        try:
            with open("books.txt", "r") as book_records:
                for line in book_records:
                    if title in line:
                        return True
            return False
        except FileNotFoundError:
            return False

    def update(self, book_id: str, title: str, author: str, availability: bool) -> bool:
        """
        Update the specified book log.
        Desc:
            This method update book log based on its specified title.
        Params & Args:
            book_id -- book identification number (str)
            title -- book title (str)
            author -- book author (str)
            availabilty -- book availability (bool value)
        Returns:
            True if operation is succeed otherwise false.
        """
        new_book = f"{book_id},{title},{author},{availability}\n"
        book_title = str(input("Book Title: "))
        with open("books.txt", "r+") as book_records:
            book_data = book_records.readlines()
            book_records.seek(0)
            for line in book_data:
                if book_title in line:
                    book_records.truncate()
                    book_records.write(new_book)
                else:
                    book_records.write(line)
            return True
        return False


    def delete(self, title: str) -> bool:
        """
        Delete the specified log in the books.txt file.
        seek() -- Moves the file pointer to a specific position within the file.
        truncate() -- Cuts off the file content from the current file pointer position onward.

        Args:
           Line1
           Line2

        Returns:
            Return statement
        """
        try:
            with open("books.txt", "r+") as book_records:
                book_data = book_records.readlines()
                book_records.seek(0)
                for line in book_data:
                    if title not in line:
                        book_records.write(line)
                book_records.truncate()
                return True
            return False
        except FileNotFoundError:
            return False


class Member(User):
    def __init__(self, user_id: str, name: str, email: str) -> None:
        super().__init__(user_id, name, email)
        self.registration()

    def borrow_book(self, title: str) -> bool:
        with open("books.txt", "r+") as book_records:
            book_data = book_records.readlines()
            book_records.seek(0)
            for line in book_data:
                if title in line:
                    book_records.truncate()
                    book_records.write(line.replace("True", "False", 1))
                else:
                    book_records.write(line)
            return True
        return False


    def return_book(self, title: str) -> bool:
        with open("books.txt", "r+") as book_records:
            book_data = book_records.readlines()
            book_records.seek(0)
            for line in book_data:
                if title in line:
                    book_records.truncate()
                    book_records.write(line.replace("False", "True", 1))
                else:
                    book_records.write(line)
            return True
        return False

class Book:
    def __init__(self, book_id: str, title: str, author: str, availability: bool):
        self.book_id = book_id
        self.title = title
        self.author = author
        self.availability = availability

    def display(self):
        print(f"Book Title: {self.title}")
        print(f"Book Author: {self.author}")


class LibraryManager():
    def __init__(self, user_id: str, name: str, email: str) -> None:
        self.librarian = Librarian(user_id, name, email)


    def add(self, book_id: str, title: str, author: str, availability: bool = True) -> None:
        if self.librarian.add(book_id, title, author, availability):
            print(f"{title} added sucessfully!")
        else:
            print(f"{title} already exits!")

    def update(self, book_id: str, title: str, author: str, availability: bool) -> None:
        if (self.librarian.update(book_id, title, author, availability)):
            print(f"{title}, updated successfully!")
        else:
            print(f"{title}, not found.")


    def delete(self):
        title = str(input("Book Title: "))
        if (self.librarian.delete()):
            print(f"{title}, removed successfully!")
        else:
            print(f"{title}, not found.")

    def borrow_book(self):
        title = str(input("Book Title: "))
        if (Member.borrow_book(title)):
            print("Book borrowed.")
        else:
            print(f"{title}, not found.")

    def return_book(self):
        title = str(input("Book Title: "))
        if (Member.return_book(title)):
            print("Book returned.")
        else:
            print("{title}, not found.")

    def user_registration(self, user_id: str, name: str, email: str):
        new_user = User(user_id, name, email)
        if(new_user.validate_user_email(email)):
            new_user.registration()
            print(f"{name}, registered successfully!")
        else:
            print(f"{email}, not valid.")

    def display_all_book(self):
        with open("books.txt", "r+") as book_records:
            for line in book_records:
                print(line)


ali = LibraryManager("1043", "Ali Hassan", "alihassan@gmail.com")
ali.add("1001", "The Prince", "Nicolo Machivali")
ali.add("1002", "Khud Say Khuda Tak", "Nair Iftikhar")
ali.add("1003", "Prisoner", "Charles Dicken")
ali.user_registration("1043", "Mubeen", "mubeenulhassan@gmail.com")
ali.display_all_book()
